# Data structures

**Author(s)**: Jukka Aho

**Abstract**: Description of basis data structures. In this notebook the concepts of `Increment`, `TimeStep`, `Field`, `FieldSet`, `SpatialBasis`, `TemporalBasis` are intoduced. With combining these atomic structures one is able to form finite elements and interpolate it's fields in time and spatial domain.

- `Increment` is the most atomic structure. It's a vector-like object with 1 dimension. Each element in `Increment` can be scalar, vector or tensor (2 or 4 order). It's easy to extend `Increment` to have other data types too.
- `TimeStep` is container for increments in certain time $t$.
- `Field` is container for timesteps for a single field.
- `FieldSet` is container for all fields.

## Revision history

### 2015-06-14
- Initial version.

### 2015-09-25
- Complete rewrite. The main ideas proposed in earlier version didn't work.

### 2015-10-29
- Third iteration.

### Data fields on elements

Typical element structure so far:

    type MyElement <: Element
        connectivity :: Array{Int, 1}  # describes how dofs of this element is connected to another elements in global level
        basis :: Basis  # describes how to interpolate fields
        fields :: ???
    end

- Fields must be interpolable, in space $\mathbb{C}^n \times \mathbb{R}$, i.e. $f(\boldsymbol{\xi}, t) = \sum_i \phi_i(\boldsymbol{\xi}) f_i(t) = \sum_i \phi_i(\boldsymbol{\xi}) \sum_j \varphi_j(t) f_{ij}$, where $f_{ij}$ is scalar, tensor or vector defined in element area $e$ by some basis functions $\phi(\boldsymbol{\xi})$ and $\varphi(t)$. Parameter $t$ is normally considered as "time" and $\xi$ is dimensionless coordinate. Parameter $t$ has not necessarily to be time, it could be for example angle $\alpha \in [-2\pi, 2\pi]$ or similar.
- We store mainly three fields, scalar field, vector field, tensor field. Field may or may not be dependent from parameters $\xi$ or $t$.
- $t$ is discretized to several steps $\{t_0, t_1, \ldots, t_n\}$. Each discrete time $t_i$ may contain several iterations until convergence. We want to save and get access to all of this data if needed.
- So in practice we have a set of fields $f(\boldsymbol{\xi})$ over time domain $t$. Typically some fields, like Geometry, is introduced only in time $t_0$. Some other fields, like boundary load, may be "active" only on some time $\hat{t} \subset t$. Some care must be taken of how to extrapolate field variables.
- In the simplest case (simple nonlinear quasistatic analysis), we have for instance $t \in [0, 1]$ where boundary conditions are set in $t_0$ and load is set in $t_1$. We may use adaptive strategies to shorten time if convergence issues araises.

Here's the strategy in short: each non-linear iteration is `Increment`, what is a vector-like object containing data. Elements of `Increment` can be scalars, vectors or tensors. `Increment` belongs to `TimeStep`. `TimeStep` contains one or more increments. Then we have `Field` which contains all timesteps. And finally we have `FieldSet` which contains all fields.

    FieldSet -> Field -> TimeStep -> Increment -> data

for example,

    FieldSet -> Field -> TimeStep -> Increment -> data
    FieldSet -> "temperature" -> 0.0 -> 1 -> [1,2,3,4]
                                        2 -> [2,3,4,5]
    FieldSet -> "temperature" -> 1.0 -> 1 -> [3,4,5,6]
                                        2 -> [5,6,7,8]

and so on.

### Creating discrete fields

Here we create a `FieldSet` containing one field `temperature` which contains two `TimeStep`s, two `Increment`s in both of them.

In [1]:
using JuliaFEM: Increment, TimeStep, Field, FieldSet

fs = FieldSet()
i1 = Increment([1, 2, 3])
i2 = Increment([2, 3, 4])
t1 = TimeStep(1.0, Increment[i1, i2])
i3 = Increment([2, 3, 4])
i4 = Increment([3, 4, 5])
t2 = TimeStep(2.0, Increment[i3, i4])
f1 = Field(TimeStep[t1, t2])
fs["temperature"] = f1
fs

Dict{ASCIIString,JuliaFEM.AbstractField} with 1 entry:
  "temperature" => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[JuliaFEM.Inc…

Accessing last increment of last timestep of temperature can be done in the following way:

In [2]:
increment = fs["temperature"][end][end]
increment, typeof(increment)

([3,4,5],JuliaFEM.Increment{Int64})

Because the model is deep and quite heavy to type some "shortcuts" are provided, but keep in mind that everything is there in place. Here's the shortened version:

In [3]:
fs = FieldSet()  # create empty fieldset
fs["temperature"] = [1, 2, 3, 4]  # create temperature field with time t=0
first(fs["temperature"]), last(fs["temperature"])  # pick first and last timestep of temperature

([1,2,3,4],[1,2,3,4])

This way we can easily (discrete) create scalar, vector and tensor fields.

In [4]:
fs2 = FieldSet()
fs2["constant scalar field"] = 1
fs2["scalar field"] = [1, 2, 3, 4]
fs2["vector field"] = reshape(collect(1:8), 2, 4)
fs2["second order tensor field"] = reshape(collect(1:3*3*4), 3, 3, 4)
fs2["fourth order tensor field"] = reshape(collect(1:3*3*3*3*4), 3, 3, 3, 3, 4)
fs2

Dict{ASCIIString,JuliaFEM.AbstractField} with 5 entries:
  "fourth order tensor fi… => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "constant scalar field"  => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "vector field"           => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "scalar field"           => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "second order tensor fi… => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…

Or even more compactly:

In [5]:
FieldSet("geometry" => [1, 2, 3, 4], "temperature" => [0, 0, 0, 0], "density" => 7850)

Dict{ASCIIString,JuliaFEM.AbstractField} with 3 entries:
  "density"     => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[JuliaFEM.Inc…
  "geometry"    => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[JuliaFEM.Inc…
  "temperature" => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[JuliaFEM.Inc…

By default using this "fast typing" fields are defined at time $t=0.0$.

In [6]:
fs2["vector field"][end].time  # pick last timestep of field "vector field"

0.0

Creating new field with several time steps defined can also be done compactly. Each tuple has time and increment data.

In [7]:
fs2["time series"] = (0.0, [1, 2, 3, 4]), (0.5, [2, 3, 4, 5]), (1.0, [1, 1, 1, 1])
fs2["time series"][end].time
fs2

Dict{ASCIIString,JuliaFEM.AbstractField} with 6 entries:
  "fourth order tensor fi… => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "constant scalar field"  => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "vector field"           => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "scalar field"           => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "time series"            => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "second order tensor fi… => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…

Or even without explicitly expressing time. In that case time step size is 1 second starting from 0.

In [8]:
fs2["time series 2"] = [1, 2, 3, 4], [2, 3, 4, 5], [1, 1, 1, 1]
fs2

Dict{ASCIIString,JuliaFEM.AbstractField} with 7 entries:
  "fourth order tensor fi… => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "constant scalar field"  => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "vector field"           => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "scalar field"           => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "time series"            => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "time series 2"          => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…
  "second order tensor fi… => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[J…

To add another field, with different time.

In [9]:
T0 = first(fs["temperature"])  # pick first timestep (or to be spesific, last increment of first timestep)
T1 = T0 + 1  # create new increment from old one
timestep = TimeStep(1.0, T1)  # create new timestep at t=1.0
push!(fs["temperature"], timestep)  # push to field

2-element Array{JuliaFEM.TimeStep{T},1}:
 JuliaFEM.Increment[[1,2,3,4]]
 JuliaFEM.Increment[[2,3,4,5]]

Normal stuff like dot product works:

In [10]:
S1 = Increment([1, 2, 3])
S2 = Increment([2, 3, 4])
dot(S1, S2), dot([1, 2, 3], S2), dot(S1, [2, 3, 4])

(20,20,20)

In [11]:
1/2*(S1 + S2)

3-element Array{Float64,1}:
 1.5
 2.5
 3.5

In [12]:
dot([1, 2], Increment[S1, S2])

3-element Array{Int64,1}:
  5
  8
 11

Creating empty `Increment`:

In [13]:
f = zeros(Increment, 2, 4)

4-element JuliaFEM.Increment{Array{Float64,1}}:
 [0.0,0.0]
 [0.0,0.0]
 [0.0,0.0]
 [0.0,0.0]

Create similar increment with new data:

In [14]:
g = similar(f, ones(8))

4-element JuliaFEM.Increment{Array{Float64,1}}:
 [1.0,1.0]
 [1.0,1.0]
 [1.0,1.0]
 [1.0,1.0]

### Creating continuous and discrete fields

In the last section the concept of fields was demonstrated. The `Field` is actually just a typealias for a `DefaultDiscreteField` and `DefaultDiscreteField` is subtype of `DiscreteField` which is subtype of `AbstractField`:

In [15]:
using JuliaFEM: AbstractField, DiscreteField
Field <: DiscreteField <: AbstractField

true

There exists another type of fields too, namely `ContinuousField`s. Like the name already suggests it stores continuous time and spatial domain and it can be used to write custom fields. It needs to be callable. In this example `ContinuousField` is created which returns 1x4 dimensional array defined in $\boldsymbol\xi \in [-1,1]^2, t\in[0,1]$:

In [16]:
using JuliaFEM: FieldSet, ContinuousField

In [17]:
type MyFunnyField <: ContinuousField
end
function Base.call(f::MyFunnyField, xi::Vector, time::Number)
    time/4*[(1-xi[1])*(1-xi[2]) (1+xi[1])*(1-xi[2]) (1+xi[1])*(1+xi[2]) (1-xi[1])*(1+xi[2])]
end
f = MyFunnyField()

MyFunnyField()

In [18]:
f([0.0, 0.0], 1.0)

1x4 Array{Float64,2}:
 0.25  0.25  0.25  0.25

In [19]:
fs = FieldSet()
fs["basis"] = MyFunnyField()
fs

Dict{ASCIIString,JuliaFEM.AbstractField} with 1 entry:
  "basis" => MyFunnyField()

In [20]:
fs["basis"]([0.0, 0.0], 1.0)

1x4 Array{Float64,2}:
 0.25  0.25  0.25  0.25

Naturally we can pass another fields or even fieldsets to continuous field to make fields depend from each other. Here's another example, where `ContinuousField` takes another field and operates it with some function.

In [21]:
type MyFunnyContinuousField <: ContinuousField
    basis :: Function
    discrete_field :: DiscreteField
end
function Base.call(field::MyFunnyContinuousField, xi::Vector, time::Number=1.0)
    data = last(field.discrete_field) # get the last timestep last increment
    basis = field.basis(xi) # evaluate basis at point ξ.
    sum([basis[i]*data[i] for i=1:length(data)]) # sum results
end

call (generic function with 1246 methods)

Now we create two fields, one is discrete and another is continuous which takes discrete field as parametes

In [22]:
fs = FieldSet()
fs["discrete field"] = [1, 2, 3, 4]
basis(xi) = 1/4*[
    (1-xi[1])*(1-xi[2]),
    (1+xi[1])*(1-xi[2]),
    (1+xi[1])*(1+xi[2]),
    (1-xi[1])*(1+xi[2])]
fs["continuous field"] = MyFunnyContinuousField(basis, fs["discrete field"])

MyFunnyContinuousField(basis,JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[JuliaFEM.Increment[[1,2,3,4]]]))

Results:

In [23]:
fs["continuous field"]([0.0, 0.0], 1.0), last(fs["discrete field"])

(2.5,[1,2,3,4])

Add another discrete field:

In [24]:
T0 = last(fs["discrete field"])
push!(fs["discrete field"], TimeStep(1.0, T0 + 1.0))  # push to field
fs

Dict{ASCIIString,JuliaFEM.AbstractField} with 2 entries:
  "continuous field" => MyFunnyContinuousField(basis,JuliaFEM.DefaultDiscreteFi…
  "discrete field"   => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[JuliaFE…

Updated results, notice how the value of continuous field changes according to the update of discrete field.

In [25]:
fs["continuous field"]([0.0, 0.0], 1.0), last(fs["discrete field"])

(3.5,[2,3,4,5])

What we did is that we actually interpolated discrete field using continuous functions. We evaluated discrete field using bilinear basis at midpoint of "element":

In [26]:
1/4*(1+2+3+4), 1/4*(2+3+4+5)

(2.5,3.5)

From continuous field we can also go back to discrete fields, if needed. Here we evaluate continuous field in four discrete points, let's call them to Gauss quadrature points.

In [27]:
using JuliaFEM: DiscreteField
type MyFunnyDiscreteField <: DiscreteField
    discrete_points :: Vector
    continuous_field :: ContinuousField
end
Base.length(field::MyFunnyDiscreteField) = length(field.discrete_points)
Base.endof(field::MyFunnyDiscreteField) = endof(field.discrete_points)
Base.last(field::MyFunnyDiscreteField) = Float64[field[i] for i=1:length(field)]
function Base.getindex(field::MyFunnyDiscreteField, idx::Int64)
    field.continuous_field(field.discrete_points[idx])
end

getindex (generic function with 126 methods)

In [28]:
discrete_points = 1.0/sqrt(3.0)*Vector[[-1, -1], [1, -1], [1, 1], [-1, 1]]
fs["discrete field 2"] = MyFunnyDiscreteField(discrete_points, fs["continuous field"])
last(fs["discrete field 2"])

4-element Array{Float64,1}:
 2.75598
 3.08932
 3.91068
 4.24402

Basically summing the above values together we have just done numerical integration over element area.  By using these two simple concepts we are able to construct very interesting results.

## Interpolation

In earlier the concepts of `DiscreteField` and `ContinuousField` were introduced, so that now we can define discrete set of values and continuous functions. It has also been shown how fields can depend from each other such a way that we interpolate continuous field from discrete field and vice versa. 

This motivates us to create continuous fields which are interpolated from discrete values with some proper basis. By thinking this way interpolation is nothing more than just an application of the earlier results already shown. 

Interpolation of fields works of course both in time and spatial dimension. Here's a simple example showing the main concept:

In [29]:
using JuliaFEM: TemporalBasis, SpatialBasis

In [30]:
# first unanonymous function is the actual basis and second one is derivative with respect to time
temporalbasis = TemporalBasis((t) -> [1-t, t], (t) -> [-1, 1])

basis(xi) = 1/4*[(1-xi[1])*(1-xi[2])   (1+xi[1])*(1-xi[2])   (1+xi[1])*(1+xi[2])   (1-xi[1])*(1+xi[2])]
dbasis(xi) = 1/4*[
    -(1-xi[2])    (1-xi[2])   (1+xi[2])  -(1+xi[2])
    -(1-xi[1])   -(1+xi[1])   (1+xi[1])   (1-xi[1])]
spatialbasis = SpatialBasis(basis, dbasis)

temporalbasis(0.2)

2-element Array{Float64,1}:
 0.8
 0.2

In [31]:
spatialbasis([0.0, 0.0])

1x4 Array{Float64,2}:
 0.25  0.25  0.25  0.25

### Interpolation in time domain

To interpolate in time domain, call `DefaultDiscreteField` with `TemporalBasis` and time. Result is a `Increment` interpolated to that time. Here we interpolate the position of particle moving $x = \frac{1}{2}t^2$ at time $t=1.0$.

In [32]:
fs = FieldSet()
t = collect(linspace(0, 2, 5))
x = 1/2*t.^2
t, x

([0.0,0.5,1.0,1.5,2.0],[0.0,0.125,0.5,1.125,2.0])

In [33]:
x2 = tuple(collect(zip(t, x))...)

((0.0,0.0),(0.5,0.125),(1.0,0.5),(1.5,1.125),(2.0,2.0))

In [34]:
fs["particle position"] = x2
temporalbasis = TemporalBasis((t) -> [1-t, t], (t) -> [-1, 1])
call(fs["particle position"], temporalbasis, 1.0)

1-element Array{Float64,1}:
 0.5

It's also possible to take time derivatives. To do so, call `Field` with `TemporalBasis`, time, and additional argument `Val{:derivative}`. Again, same example:

### Interpolation in spatial domain

To interpolate in spatial domain, call `Increment` with `SpatialBasis` and coordinate $\boldsymbol\xi$. Increments to interpolate are the latest ones in each time step. Result depends from the content of the field. If it is scalar field, result will be scalar, if it's vector the result will be vector and so on.

Let's have a $\left[0,1\right]\times\left[0,1\right] \in \mathbb{R}^2$ domain and $u_5 = 0.25$ displacement in upper right corner pointint to the $x_1$ direction. We seek for a center point of this at time $t=1.0$.

In [35]:
fs = FieldSet()
fs["geometry"] = Vector{Float64}[[0.0,0.0], [1.0,0.0], [1.0,1.0], [0.0,1.0]]
fs["displacement"] = (0.0, zeros(2, 4)), (1.0, Vector[[0.0, 0.0], [0.0, 0.0], [0.25, 0.0], [0.0, 0.0]])
fs

Dict{ASCIIString,JuliaFEM.AbstractField} with 2 entries:
  "geometry"     => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[JuliaFEM.In…
  "displacement" => JuliaFEM.DefaultDiscreteField(JuliaFEM.TimeStep[JuliaFEM.In…

In [36]:
X = call(last(fs["geometry"]), spatialbasis, [0.0, 0.0])
u = call(last(fs["displacement"]), spatialbasis, [0.0, 0.0])
x = X+u
x

2-element Array{Float64,1}:
 0.5625
 0.5   

It's also possible to take gradient of field. To calculate gradient, call `Increment` with `SpatialBasis` and coordinate $\boldsymbol\xi$. Also, give another `Increment` to calculate Jacobian, typically geometry, 
and add additional argument `Val{:gradient}'.

In [37]:
gradu = call(last(fs["displacement"]), spatialbasis, [0.0, 0.0], last(fs["geometry"]), Val{:gradient})

2x2 Array{Float64,2}:
 0.125  0.125
 0.0    0.0  

Now we have:
- `FieldSet` which defines discrete values in time and space
- Interpolants `TemporalBasis` and `SpatialBasis` which defines how discrete fields values are interpolated to get continuous fields.

Let's plug these in one new composite type and call it to *Finite Element*:

In [38]:
abstract AbstractElement

type FiniteElement <: AbstractElement
    connectivity :: Array{Int, 1}
    basis :: SpatialBasis
    time :: TemporalBasis
    fields :: Dict{ASCIIString, FieldSet}
end

function FiniteElement(connectivity)
    f(t) = [1-t, t]
    df(t) = [-1, 1]
    temporal_basis = TemporalBasis(f, df)

    h(xi) = 1/4*[(1-xi[1])*(1-xi[2])   (1+xi[1])*(1-xi[2])   (1+xi[1])*(1+xi[2])   (1-xi[1])*(1+xi[2])]
    dh(xi) = 1/4*[
        -(1-xi[2])    (1-xi[2])   (1+xi[2])  -(1+xi[2])
        -(1-xi[1])   -(1+xi[1])   (1+xi[1])   (1-xi[1])]
    spatial_basis = SpatialBasis(h, dh)

    return FiniteElement(connectivity, spatial_basis, temporal_basis, Dict())
end

FiniteElement

In [39]:
fe = FiniteElement([1, 2, 3, 4])
fe["geometry"] = Vector{Float64}[[0.0,0.0], [1.0,0.0], [1.0,1.0], [0.0,1.0]]
fe["displacement"] = (0.0, zeros(2, 4)), (1.0, Vector[[0.0, 0.0], [0.0, 0.0], [0.25, 0.0], [0.0, 0.0]])

basis = get_basis(fe)
basis("geometry", [0.0, 0.0])

dbasis = grad(basis)
dbasis("displacement", [0.0, 0.0], 0.5)
dbasis("displacement", [0.0, 0.0], 1.0)

u = fe["displacement"]
strain = 1/2(grad(u) + grad(u)')
strain_rate = diff(strain)
strain_rate([0.0, 0.0], 1.0)

LoadError: LoadError: MethodError: `setindex!` has no method matching setindex!(::FiniteElement, ::Array{Array{Float64,1},1}, ::ASCIIString)
while loading In[39], in expression starting on line 2